In [45]:
import pandas as pd
import re
import numpy as np

class POSReport:
    def __init__(self, file_path, header_row_index, columns_to_keep):
        self.file_path = file_path
        self.header_row_index = header_row_index
        self.columns_to_keep = columns_to_keep
        self.output_table = []
        self.df = pd.read_excel(self.file_path, header= self.header_row_index, usecols=self.columns_to_keep, dtype='str')


class InventoryList(POSReport):
    def __init__(self, file_path, header_row_index=4, columns_to_keep="A:C"):
        super().__init__(file_path, header_row_index, columns_to_keep)
        self.filter_df()

    def filter_df(self):
        """
        Filter Steps:
        - Disregard contents in columns D onwards
        - Remove blank rows
        - Remove irrelevat rows (non-item rows) such as column, category, total rows
        - Remove Duplicate Rows
        """
        self.df.dropna(thresh=3, inplace=True)
        # Drop Header Rows and Total Rows
        self.df = self.df.drop(self.df[(self.df.Category == 'Category') & (self.df.Name == 'Name') & (self.df.UPC == 'UPC')].index)
        self.df = self.df.drop(self.df[(self.df.Category == 'None') & (self.df.Name == 'Open Item - Transaction Fee')].index)
        self.df = self.df.drop(self.df[(self.df.Category == 'None') & (self.df.Name == 'Transaction Fee')].index)
        # Drop duplicate rows
        self.df.drop_duplicates(keep='last', inplace=True)
        return
        

class PriceList(POSReport):
    def __init__(self, file_path, header_row_index=3, columns_to_keep='A:C'):
        super().__init__(file_path, header_row_index, columns_to_keep)
        self.filter_df()

    def filter_df(self):
        """
        Filter Steps:
        - Disregard contents in columns D through G
        - Remove blank rows
        - Remove irrelevat rows (non-item rows) such as column, category, total rows
        - Strip/Upper each item name & size
        - Try and extract the unit/packaging size
        - Remove Duplicate Rows
        """
        self.df.rename(columns={"Item Name": "Name"}, inplace=True)
        # Drop blank rows
        self.df.dropna(thresh=2, inplace=True, subset=['Name', 'Price'])
        # Drop Header Rows and Total Rows
        self.df = self.df.drop(self.df[(self.df.Name == 'Item Name') & (self.df.Size == 'Size') & (self.df.Price == 'Price')].index)
        self.df = self.df.drop(self.df[(self.df.Name == 'None') & (self.df.Price == 'Open Item - Transaction Fee')].index)
        self.df = self.df.drop(self.df[(self.df.Name == 'None') & (self.df.Price == 'Transaction Fee')].index)
        # Drop duplicate rows
        self.df.drop_duplicates(keep='last', inplace=True)
        return
           

class SalesList(POSReport):
    def __init__(self, file_path, header_row_index=7, columns_to_keep='A:B'):
        super().__init__(file_path, header_row_index, columns_to_keep)
        self.filter_df()

    def filter_df(self):
        """
        Extract only the items and their UPC on the sales list
        """
        self.df.rename(columns={"Item Name": "Name"}, inplace=True)
        self.df.dropna(thresh=2, inplace=True)
        # Drop Header Rows and Total Rows
        self.df = self.df.drop(self.df[(self.df.UPC == 'UPC') & (self.df.Name == 'Item Name') & (self.df.UPC == 'UPC')].index)
        # Drop duplicate rows
        self.df.drop_duplicates(keep='last', inplace=True)
        return
        
pd.set_option('display.max_rows', 500)

In [46]:
inventory = InventoryList('./pos_reports/inventory/inv_10_14_2021.xls')
price = PriceList('./pos_reports/price/price_10_14_2021.xls')

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [47]:
# Get a df of all items sold in the past 6 months
sales_dfs = []
# Read in and append SalesList objects to the list
for i in range(4,10):
    sales_dfs.append(SalesList(f'./pos_reports/sales/sales_{i}.xls').df)

all_sales_items = pd.concat(sales_dfs)
all_sales_items = all_sales_items.drop_duplicates(keep='last')
print(len(all_sales_items))

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
3278


In [48]:
print(len(inventory.df))
print(len(all_sales_items))

6430
3278


In [49]:
# Remove inactive items
active_inventory = pd.merge(all_sales_items, inventory.df, on=['Name','UPC'], how='right', indicator='Active')
active_inventory['Active'] = np.where(active_inventory.Active == 'both', True, False)
active_inventory = active_inventory[active_inventory['Active']==True]
active_inventory.drop("Active", axis=1, inplace=True)

In [50]:
# Create a column that marks items with duplicates (need price manually changed)
active_inventory['Duplicate'] = active_inventory['Name'].duplicated()

In [51]:
print(len(active_inventory))

3273


In [52]:
complete_list = pd.merge(active_inventory, price.df,  how='left', on='Name', validate='many_to_many')
complete_list.drop_duplicates(subset=['UPC', 'Name'], keep='first', inplace=True)
complete_list.to_excel('product_list.xls')

C:\Users\Yuta\AppData\Local\Temp/ipykernel_1104/546642087.py:3: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  complete_list.to_excel('product_list.xls')


In [53]:
unit_expression = '\d+\.*\d* *(ML|L|PAK|PK|OZ|GALLON|GAL|CAN|OZ|L|P|O|BLT|BTL|PACK|CT|0Z|BTLS|OZ BOTTLE|CN|G|LBS|QT|Z|STICKS|OZ CN|OZ CAN){1} *$'

def extract_unit_size(x):
    item_name = str(x)
    filter = re.compile(unit_expression)
    if re.search(filter, item_name):
        unit_size = item_name[re.search(filter, item_name).start():re.search(filter, item_name).end()].strip()
        return re.sub(r"([0-9]+(\.[0-9]+)?)",r"\1 ", unit_size)
    else:
        return ''

def remove_unit_size(x):
    item_name = str(x)
    filter = re.compile(unit_expression)
    if re.search(filter, item_name):        
        return (item_name[:re.search(filter, item_name).start()] + item_name[re.search(filter, item_name).end()+1:]).strip()
    else:
        return item_name

In [54]:
complete_list.Name = complete_list.Name.apply(lambda x: x.upper())
complete_list.Size = complete_list.Name.apply(lambda x: extract_unit_size(x))
complete_list.Name = complete_list.Name.apply(lambda x: remove_unit_size(x))


In [55]:
complete_list.to_excel('product_list.xls')

C:\Users\Yuta\AppData\Local\Temp/ipykernel_1104/154801814.py:1: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  complete_list.to_excel('product_list.xls')
